In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [25]:
url = 'https://quotes.toscrape.com'
r = requests.get(url)

if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')
    
    body = soup.body
    
    for quote in soup.find_all('div',class_='quote'):
 
        text = quote.find('span', class_='text').text
        author = quote.find('small', class_='author').text
        tags = quote.find('div',class_='tags')
        alltags = []
        for ntag in tags.find_all('a'):
            alltags.append(ntag.text)
        print(f'{text} \n  author: {author} \n\n tags:{alltags}')
    # print('successfully scraped',quote_div)
else:
    print(r)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” 
  author: Albert Einstein 

 tags:['change', 'deep-thoughts', 'thinking', 'world']
“It is our choices, Harry, that show what we truly are, far more than our abilities.” 
  author: J.K. Rowling 

 tags:['abilities', 'choices']
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” 
  author: Albert Einstein 

 tags:['inspirational', 'life', 'live', 'miracle', 'miracles']
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” 
  author: Jane Austen 

 tags:['aliteracy', 'books', 'classic', 'humor']
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” 
  author: Marilyn Monroe 

 tags:['be-yourself', 'inspirational']
“Try not to become a man of success. Rather become a man of value.” 
  auth